In [4]:
import tensorflow as tf

# tf.keras.applications.resnet50.preprocess_input

In [16]:
# Load the model from the checkpoint
model = tf.keras.models.load_model(
    "/home/ubuntu/dev/carni_detect/model_checkpoint.keras"
)

image = tf.keras.preprocessing.image.load_img(
    "/home/ubuntu/dev/carni_detect/backup_data/Dog-Breeds-Dataset/bosnian broken-haired hound - called barak dog/Image_1.jpg",
    target_size=(224, 224),
)
image = tf.keras.preprocessing.image.img_to_array(image)
image = tf.expand_dims(image, axis=0)
image = tf.keras.applications.resnet50.preprocess_input(image)

# Make predictions
predictions = model.predict(image)
print("Predictions:", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predictions: [[0.001 0.    0.    0.01  0.    0.    0.    0.    0.881 0.104 0.004 0.
  0.    0.    0.   ]]


In [22]:
def preprocess_dataset(dataset: tf.data.Dataset):
    dataset = dataset.map(
        lambda image, label: (
            tf.keras.applications.resnet50.preprocess_input(image),
            label,
        )
    )
    return dataset


def get_image_dataset(
    data_directory: str, batch_size: int, shuffle: bool
) -> tf.data.Dataset:
    """Creates a TensorFlow dataset from the images in the specified directory.

    Args:
        data_directory (str): Path to the directory containing the image files.
        batch_size (int): Number of images to include in each batch.

    Returns:
        tf.data.Dataset: A TensorFlow dataset object.
    """
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_directory,
        label_mode="categorical",
        seed=42,
        batch_size=batch_size,
        image_size=(224, 224),  # Resize images to a consistent size
        shuffle=shuffle,
        color_mode="rgb",
    )
    dataset = preprocess_dataset(dataset)
    return dataset


test_dataset = get_image_dataset(
    data_directory="/home/ubuntu/dev/carni_detect/dataset/test",
    batch_size=32,
    shuffle=False,
)

Found 375 files belonging to 15 classes.


In [23]:
pred = model.evaluate(test_dataset)
print(pred)

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step - categorical_accuracy: 0.9493 - loss: 0.1795
[0.17947760224342346, 0.9493333101272583]


In [17]:
import numpy as np

np.set_printoptions(precision=3, suppress=True)
print("Predictions:", predictions)

Predictions: [[0.001 0.    0.    0.01  0.    0.    0.    0.    0.881 0.104 0.004 0.
  0.    0.    0.   ]]


In [18]:
final_pred = tf.argmax(predictions, axis=1)
print(final_pred)

tf.Tensor([8], shape=(1,), dtype=int64)
